In [1]:
TaskObject = ""

StatementMeta(arkstgsynspads, 10, 1, Finished, Available)

SyntaxError: EOL while scanning string literal (<stdin>, line 1)

In [4]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

session_id = random.randint(0,1000000)

TaskObjectJson = json.loads(TaskObject)
Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
Schema = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"


Source = Source + TaskObjectJson['Source']['RelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
Schema = Schema + TaskObjectJson['Source']['RelativePath'] + "/" + TaskObjectJson['Source']['SchemaFileName']
Target = Target + TaskObjectJson['Target']['RelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']


#remove any double slashes
Source = Source.replace('//', '/')
Schema = Schema.replace('//', '/')
Target = Target.replace('//', '/')

#get source and target data types
SourceDT = TaskObjectJson['Source']['Type']
TargetDT = TaskObjectJson['Target']['Type']


#add abfss
Source = "abfss://" + Source
Schema = "abfss://" + Schema
Target = "abfss://" + Target


print ("Source: " + Source)
print ("Schema: " + Schema)
print ("Target: " + Target)



StatementMeta(dlzstgsynspads, 82, 4, Finished, Available)

Source: abfss://datalakeraw@dlzstgdlsadsylf2adsl.dfs.core.windows.net/SampleFiles/SalesLT.Customer.chunk_1.parquet
Schema: abfss://datalakeraw@dlzstgdlsadsylf2adsl.dfs.core.windows.net/SampleFiles/SalesLT.Customer.json
Target: abfss://datalakelanding@dlzstgdlsadsylf2adsl.dfs.core.windows.net/TestOutput/SalesLT-Customer-Delta

In [5]:
#get scheme file ->  this needs to be made dynamic
schema = spark.read.load(Schema, format='json', multiLine=True)
#convert it into a list so we can loop it using python rules
schema = schema.collect()
#loop through each column to find the primary key column
for col in schema:
    if col.PKEY_COLUMN:
        print(col.COLUMN_NAME)
        primaryKey = col
        break
#set up the merge condition used in the next code block
mergeCondition = "oldData." + primaryKey.COLUMN_NAME + " = newData." + primaryKey.COLUMN_NAME

StatementMeta(dlzstgsynspads, 82, 5, Finished, Available)

CustomerID

In [36]:
from delta.tables import *
if(TargetDT == 'Delta'):
    if(SourceDT == 'Parquet'):
        print("SourceDT = Parquet, TargetDT = Delta.")
        df = spark.read.load(Source, format='parquet')
    else:
        print("SourceDT = Delta, TargetDT = Delta.")
        df = spark.read.load(Source, format='delta')
    sql = 'describe detail "' + Target + '"'
    try:
        if (spark.sql(sql).collect()[0].asDict()['format'] == 'delta'):
            print("Table already exists. Performing Merge")
            olddt = DeltaTable.forPath(spark, Target)  
            olddt.alias("oldData").merge(
                df.alias("newData"),
                mergeCondition) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
        else:
            print("Table does not exist. No error, creating new Delta Table.")    
            df.write.format("delta").save(Target)
    except:
        print("Table does not exist. Creating new Delta Table.")    
        df.write.format("delta").save(Target)
elif(TargetDT == 'Parquet' and SourceDT == 'Delta'):
    print("SourceDT = Delta, TargetDT = Parquet.")
    df = spark.read.format("delta").load(Source)
    df.write.format("parquet").mode("overwrite").save(Target) 
    

StatementMeta(dlzstgsynspads, 77, 36, Finished, Available)

Table already exists. Performing Merge

In [43]:
#olddt.history().show(20, 1000, False)
#display(spark.read.format("delta").load(Target))
#spark.sql("CREATE TABLE SalesLTCustomer USING DELTA LOCATION '{0}'".format(TargetFile))

StatementMeta(dlzstgsynspads, 77, 43, Finished, Available)

+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                                         operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                                                                                                                                    operationMetrics|
+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------

In [41]:
## Execute Upsert
#(old_deltaTable
# .alias("oldData") 
# .merge(newIncrementalData.alias("newData"), "oldData.id = newData.id")
# .whenMatchedUpdate(set = {"name": col("newData.name")})
# .whenNotMatchedInsert(values = {"id": col("newData.id"), "name":
#                                col("newData.name")})
# .execute()
#)
#
# Display the records to check if the records are Merged
#display(spark.read.format("delta").load(Target))

StatementMeta(dlzstgsynspads, 77, 41, Finished, Available)

NameError: name 'old_deltaTable' is not defined

In [8]:
#olddt.history().show(20, 1000, False)

StatementMeta(dlzstgsynspads, 82, 8, Finished, Available)

+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                                         operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                                                                                                                                    operationMetrics|
+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------

In [7]:
#########################
#NOTE -> This is an alternate way of upserting into delta table. Using manual method of getting each column required from the schema / dataframe and then creating a dictionary to use for the upsert.
#           Currently not using this however it does work. Would advise to change the script to just create a dictionary and insert that instead of creating a string to convert into a dict.
#########################
#from delta.tables import *
#df = spark.read.load(Source, format='parquet')
#updatecols = []
#insertcols = []
#for col in schema:
#    updatecols.append(col.COLUMN_NAME)
#
#for col in df.dtypes:
#    insertcols.append(col[0])
#
#creating a string to be converted to dictionary 
#note -> can easily re-write this as just a dictionary if end up using this method.
#updatestring = '{'
#insertstring = '{'
#
#Go through each column in the schema to check what needs to be updated
#for col in updatecols:
#    updatestring = updatestring + '"' + col + '": "newData.' + col +'", '
#updatestring = updatestring[:-2]
#updatestring = updatestring + '}'
#
#Go through the new data to check what columns need to be inserted
#for col in insertcols:
#    insertstring = insertstring + '"' + col + '": "newData.' + col +'", '
#insertstring = insertstring[:-2]
#insertstring = insertstring + '}'
#
#print(updatestring)
#print(insertstring)
#
#convert to dict
#res = json.loads(updatestring)
#res2 = json.loads(insertstring)
#
#sql = 'describe detail "' + Target + '"'
#try:
#    if (spark.sql(sql).collect()[0].asDict()['format'] == 'delta'):
#        print("Table already exists. Performing Merge")
#        olddt = DeltaTable.forPath(spark, Target)  
#        olddt.alias("oldData").merge(
#            df.alias("newData"),
#            mergeCondition) \
#        .whenMatchedUpdate(set = res) \
#        .whenNotMatchedInsert(values = res2) \
#        .execute()
#except:
#    print("Table does not exist.")    
#    df.write.format("delta").save(Target)

StatementMeta(dlzstgsynspads, 82, 7, Finished, Available)

{"CustomerID": "newData.CustomerID", "NameStyle": "newData.NameStyle", "Title": "newData.Title", "FirstName": "newData.FirstName", "MiddleName": "newData.MiddleName", "LastName": "newData.LastName", "Suffix": "newData.Suffix", "CompanyName": "newData.CompanyName", "SalesPerson": "newData.SalesPerson", "EmailAddress": "newData.EmailAddress", "Phone": "newData.Phone", "PasswordHash": "newData.PasswordHash", "PasswordSalt": "newData.PasswordSalt", "rowguid": "newData.rowguid", "ModifiedDate": "newData.ModifiedDate"}
{"CustomerID": "newData.CustomerID", "NameStyle": "newData.NameStyle", "Title": "newData.Title", "FirstName": "newData.FirstName", "MiddleName": "newData.MiddleName", "LastName": "newData.LastName", "Suffix": "newData.Suffix", "CompanyName": "newData.CompanyName", "SalesPerson": "newData.SalesPerson", "EmailAddress": "newData.EmailAddress", "Phone": "newData.Phone", "PasswordHash": "newData.PasswordHash", "PasswordSalt": "newData.PasswordSalt", "rowguid": "newData.rowguid", "M